In [ ]:
import pandas as pd
import numpy as np
import multiprocessing as mp
import urllib.request, json

from eod import EodHistoricalData

api = EodHistoricalData("5f3afd582bd7b4.95720069")

def get_ticker_details(ticker):
    url = f"https://eodhistoricaldata.com/api/fundamentals/{ticker}?api_token=5f3afd582bd7b4.95720069&order=d&fmt=json"
    try:
        response = urllib.request.urlopen(url, timeout=5)
        data = json.loads(response.read())
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        return None
    
    if data.get('General'):
        Name = data["General"].get("Name", None)
        Country = data["General"].get("CountryName", None)
        Code = data["General"].get("Code", None)
        Exchange = data["General"].get("Exchange", None)
        Primary_ticker = ticker
        Currency = data["General"].get("CurrencyCode", None)
        Type = data["General"].get("Type", None)
        Industry = data["General"].get("Industry", None)
        Sector = data["General"].get("Sector", None)
    else:
        return None
    df = pd.DataFrame({'Name' : [Name], 
                        'Country' : [Country], 
                        'Code' : [Code], 
                        'Exchange' : [Exchange], 
                       'Primary_ticker' : [Primary_ticker], 
                       'Currency' : [Currency], 
                       'Type' : [Type], 
                       'Industry' : [Industry], 
                       'Sector' : [Sector]})
    print('Finished with current ticker: ' + ticker)
    return df

tickers = pd.read_csv("exchange_details.csv").Ticker.to_list()
sub_tickers = tickers[200000:]

if __name__ == '__main__':
    with mp.Pool(mp.cpu_count()) as pool:
        dfs_ = pool.map(get_ticker_details, sub_tickers)
    sub_combined_df = pd.concat([df for df in dfs_ if df is not None])
    sub_combined_df.to_csv("ticker_details_20.csv", index = False)